# Data Gathering

In [ ]:
%matplotlib inline

In [ ]:
from utils.io import gdf_viewer, gdf_merger, data_validation, gdf_filter, fix_duplicates
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime as dtm
import matplotlib.pyplot as plt
from definitions import ROOT_DIR

In [ ]:
def files_search(work_dir, files_dict, prefix='', skip=None, details=False):   
    
    if skip is None:
        skip = "we don't want to skip a word"
        
    for k in files_dict.keys():
        tmp_list = []
        for p,d,f in os.walk(work_dir):
            for x in f:
                add = False
                if re.search(prefix,x,re.I) and not re.search(skip,x,re.I):
                    add = True
                    i = str(f'{p}/{x}')
                else:
                    add = False
                    i=''
                    
                if re.search(k,i,re.I) and add:
                    tmp_list.append(i)
        tmp_list.sort()
        files_dict.update({k:tmp_list})

    for k,v in files_dict.items():
        print(k,' \t: ',len(v))
    
    if details: # Look filenames
        which = files_dict.keys()

        for w in which:
            print('\n+++++++++++++++++')
            print(f'+  {w.upper()}\t+ ')
            print('+++++++++++++++++')
            [print(i, '-', x) for i, x in enumerate(files_dict[w], 0)]    


In [ ]:
def create_df(files, verbose=True): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    files: list of files name
    """
    cnt = 0
    dfs = []
    for f in files:
        cnt += 1
        df = pd.read_csv(f, delimiter=',')
        dfs.append(df)
        
        if verbose:
            if 'X' in list(df.columns): msg = ' --> Coordinates'
            else: msg = ' --> No coordinates'

            print(f"df{cnt} : {msg}")
            
    return dfs

In [ ]:
def dataset_overview(d, verbose=False): # check for same datasets in given files
    """d: dict
    """
    l = len(d)
    with_coord = []
    no_coord = []
    same = []
    
    for i in range(l-1):
        for j in range(i,l):
            a, b = create_df([d[i], d[j]], verbose)
            if j != i:
                if a.equals(b):
                    same.append((i,j))
            
            if 'X' in list(b.columns) and j not in with_coord:
                with_coord.append(j)
            elif 'X' not in list(b.columns) and j not in no_coord:
                no_coord.append(j)
    
    print(f'Same files:{same}\nFiles with coordinates:{with_coord}\nFiles without coordinates:{no_coord}')

## Reading files

In [ ]:
work_dir = ROOT_DIR+'/CF_data/Result_traitem/'
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [ ]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Piezometer':0,'Piezair':0,'Trench':0,'Litho':0,'Equipm':0,
        'Measure':0,'Sample':0,'Analysis':0,'Facility':0}

In [ ]:
files_search(work_dir, files_dict, prefix='', skip='source')

In [ ]:
how=['inner', 'outer', 'left', 'right']

In [ ]:
f = False
t = True

# Boreholes

In [ ]:
file = save_dir+'Merged_Boreholes.csv'
bh = pd.read_csv(file, delimiter=',')
gdf_viewer(bh, rows=3, un_val='ID', view=t)

# Lithologies

In [ ]:
key='Litho'
save_file = f'Merged_Lithologies.csv'
coi = ['ID','X','Y','Z','Long_for','Description','Litho_top','Litho_base']
dataset = pd.DataFrame() # for saving object info after last merging
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key] #files_dict[key][0]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Drains_Pz_ENEL_Lithology.csv' # 0
file2= work_dir + 'database_Memoris3/Result_sol_Lithology.csv' # 2

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
df2.rename(columns={'Long':'Long_for', 'Description':'Description2'}, inplace=True)

In [ ]:
df2['X'] = df2['X'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
df2['Y'] = df2['Y'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
df2['Ep_remb'] = df2['Ep_remb'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

In [ ]:
df2[['X','Y','Ep_remb']] = df2[['X','Y','Ep_remb']].astype(float)

In [ ]:
df1 = df1[['ID','Date_ouv','Long_for','Description','Litho_top','Litho_base']].copy()
df2 = df2[['ID','X','Y','Z','Long_for']].copy()

In [ ]:
mdf, conflict_df = gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1., drop_skip_col=['index'])

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
gdf_viewer(conflict_df, rows=10, un_val='ID', view=t)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Profils_sol_Lithology.csv' # 1
file2= work_dir + 'donnees_terrain_2019/Log_Lithology.csv' # 3

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
for idx, row in df2.iterrows():
    if row['Type'] == 'Forage': 
        df2.loc[idx, 'Date_ouv']= '2019-10-07'
    elif row['Type'] == 'Piezo' or row['Type'] == 'Piezair': 
        df2.loc[idx, 'Date_ouv']= '2019-12-18'

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
#df2.rename(columns={'Long':'Long_for', 'Description':'Description2'}, inplace=True)

In [ ]:
df1 = df1[['ID','Date_ouv','Long_for','Description','Litho_top','Litho_base']].copy()
df2 = df2[['ID','Date_ouv','Description','Litho_top','Litho_base','Zone']].copy()

In [ ]:
mdf, conflict_df = gdf_merger(df1, df2, how=how[1], on='ID', date_col='Date_ouv', dist_max=None, drop_skip_col=['index'])

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

In [ ]:
pause # problem with dataset merging

#### Merge with object dataset

#### $\color{blue}{\textbf{Manage conflicts}}$

#### $\color{green}{\textbf{Read and merge}}$

#### $\color{blue}{\textbf{Manage conflicts}}$

#### Merge with object dataset

#### $\color{blue}{\textbf{Manage conflicts}}$

####  $\color{red}{\textbf{Save final object dataset}}$

#### -----------------------------------------------------------

## Plot

In [ ]:
plt.plot(dataset['X'],dataset['Y'], '.')
plt.axis('equal')
plt.grid('on')

In [ ]:
pause